In [1]:
#from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from collections import deque
import os
from os import listdir, mkdir
from os.path import isfile, join, isdir
import itertools
from strategies import *
from helpers import *
from joblib import Parallel, delayed

In [2]:
mypath = 'historical_data/'
countries = sorted(['Australia', 'Japan', 'China'])
fx_dict = {'Australia':('historical_data/Forex/AUD_USD_new.csv',0),
           'Japan':('historical_data/Forex/USD_JPY_new.csv',1),
           'China':('historical_data/Forex/USD_HKD_new.csv',1)}

list_pairs = []
for country in countries:
    countrypath = mypath + country
    adr_names =  [f for f in listdir(countrypath) if not isfile(join(countrypath, f))] #grab all adr names of the country
    for adr in sorted(adr_names):
        list_pairs.append((country, adr))

In [3]:
# Store ratios
for (country, adr) in list_pairs:
    ratio_geq_1, ratio = get_ratio(country, adr, fx_dict)
    if adr == "ACH_2600" or adr == "BGNE_6160":
        rounded_ratio = float('%.2g' % ratio)
    else:
        rounded_ratio = float('%.1g' % ratio)
    print("Country: {}, ADR_Stock: {}, Estimated Ratio (4 d.p.): {}, Implied Ratio (2 s.f.): {}".format(country, adr, ratio, rounded_ratio))
    ratio_df = pd.DataFrame({"ratio_geq_1" : [ratio_geq_1], "ratio" : [rounded_ratio]})
    ratio_df.to_csv(f'historical_data/{country}/{adr}/ratio.csv')
        
# This shows the empircally estimated ratio, and the implied ratio we shall assume.
# These values corroborate with the select few we checked online, like GENE_GTG and BGNE_6160.

Country: Australia, ADR_Stock: ATHE_ATH, Estimated Ratio (4 d.p.): 59.4889, Implied Ratio (2 s.f.): 60.0
Country: Australia, ADR_Stock: GENE_GTG, Estimated Ratio (4 d.p.): 595.5978, Implied Ratio (2 s.f.): 600.0
Country: Australia, ADR_Stock: IMMP_IMM, Estimated Ratio (4 d.p.): 9.9082, Implied Ratio (2 s.f.): 10.0
Country: Australia, ADR_Stock: IMRN_IMC, Estimated Ratio (4 d.p.): 39.4289, Implied Ratio (2 s.f.): 40.0
Country: Australia, ADR_Stock: JHX_JHX, Estimated Ratio (4 d.p.): 1.0043, Implied Ratio (2 s.f.): 1.0
Country: Australia, ADR_Stock: KZIA_KZA, Estimated Ratio (4 d.p.): 10.0346, Implied Ratio (2 s.f.): 10.0
Country: Australia, ADR_Stock: MESO_MSB, Estimated Ratio (4 d.p.): 5.0155, Implied Ratio (2 s.f.): 5.0
Country: Australia, ADR_Stock: PLL_PLL, Estimated Ratio (4 d.p.): 101.5788, Implied Ratio (2 s.f.): 100.0
Country: Australia, ADR_Stock: WBK_WBC, Estimated Ratio (4 d.p.): 0.9997, Implied Ratio (2 s.f.): 1.0
Country: China, ADR_Stock: ACH_2600, Estimated Ratio (4 d.p.)

In [4]:
# stock_num_per_unit is how many stocks we would buy for 1 "unit" of trade
# avg_bid_non_us_before is how much foreign currency we can buy with 1 USD, 1 minute before the Asian market opens
# avg_bid_non_us_at is how much foreign currency we can buy with 1 USD, when the Asian market opens
# avg_bid_us_before is how much foreign currency we can buy with 1 USD, 1 minute before the US market opens
# avg_bid_us_at is how much foreign currency we can buy with 1 USD, when the US market opens
# All dates are in local time: so in sequential order (for each row), it will go stock_open, stock_close, adr_open, adr_close
merged_df = data_processing(*list_pairs[1], fx_dict)
merged_df.head()

,date,adr_open,adr_close,adr_volume,stock_open,stock_close,stock_volume,avg_bid_non_us_before,avg_ask_non_us_before,avg_non_us_before,...,avg_bid_us_at,avg_ask_us_at,avg_us_at,ir,stock_num_per_unit,adr_num_per_unit,stock_open_per_unit,stock_close_per_unit,adr_open_per_unit,adr_close_per_unit
0,2015-04-10,18.08,17.36,187300,0.040,0.039,987003,1.296948,1.299612,1.298280,...,1.302409,1.305068,1.303738,NaN,600.0,1,24.0,23.4,18.08,17.36
1,2015-04-13,17.76,16.16,276200,0.039,0.039,1586945,1.303310,1.305987,1.304648,...,1.315530,1.318215,1.316872,NaN,600.0,1,23.4,23.4,17.76,16.16
2,2015-04-14,16.96,17.08,254500,0.036,0.038,2905099,1.316900,1.319606,1.318253,...,1.309820,1.312529,1.311174,NaN,600.0,1,21.6,22.8,16.96,17.08
3,2015-04-15,17.68,17.52,210600,0.039,0.038,2069419,1.312108,1.314822,1.313465,...,1.310301,1.312994,1.311647,NaN,600.0,1,23.4,22.8,17.68,17.52
4,2015-04-16,17.40,17.00,106800,0.038,0.036,2378678,1.301459,1.304133,1.302796,...,1.282035,1.284634,1.283335,NaN,600.0,1,22.8,21.6,17.40,17.00


In [5]:
def plot_ret(date, cash, country, adr, num_xticks = 15, save = False, filename = "pnl_plot.png"):
    fig = plt.figure(figsize = (20, 8))
    ax = fig.add_subplot(111)
    ax.plot(date, cash)
    plt.xticks(np.arange(0, len(date), (len(date) - 1 )// num_xticks), rotation = 30, ha = 'right', fontsize = 14)
    plt.xlim(0, len(date))
    plt.yticks(fontsize = 14)
    plt.grid(True)
    plt.title(f'PnL Chart for {adr} pair from {country}', fontsize = 18)
    if save:
        if not os.path.isdir(f'historical_data/{country}/{adr}/plots/'):
            os.mkdir(f'historical_data/{country}/{adr}/plots/')
        fig.savefig(f'historical_data/{country}/{adr}/plots/{filename}')
    else:
        plt.show();
    plt.close(fig)

# Hyperparameter Tuning

Hyperparameters:
1. Lookback window
2. Entry threshold
3. Exit threshold
4. Stop-loss threshold

Steps:
1. HP Tune each strategy for each of the pairs
2. Store results for each pair in hp_log_sfx{version}.txt
3. Store results for each strategy in results_sfx{version}.txt
4. Store best strategy for each pair in results_sfx_all.txt

In [6]:
window_grid = [30, 60, 100]
entry_grid = [1, 1.5, 2]
exit_grid = [-0.5, 0, 0.5]
stop_loss_grid = [2.5, 3, 3.5]
is_end_date = "2019-02-21"
oos_start_date = "2019-02-22"
oos_end_date = "2020-02-21"
is_start_date = "2020-02-22"

"""
HP Tune
"""

def hp_tune(pairs_trade_strategy, version, country, adr, window_grid = [30,60,100], 
            entry_grid = [1,1.5,2], exit_grid = [-0.5,0,0.5], stop_loss_grid = [2.5,3,3.5]):
    
    # Initialize variables
    hps = list(itertools.product(*[window_grid, entry_grid, exit_grid, stop_loss_grid]))
    merged_df = data_processing(country, adr, fx_dict)
    hp_log = []
    max_ret = -10000
    max_ret_hps = ()
    max_ret_port = []
    max_ret_hr = 0
    max_ret_dd = 0
    max_ret_dates = []
    
    # Go through all hyperparams
    for hp in hps:
        if version == '1' or version == '2':
            ret1, _, portfolio_values1, hits1, dates1 = pairs_trade_strategy(merged_df, 
                                                                         lookback = hp[0], 
                                                                         entry = hp[1], 
                                                                         exit = hp[2], 
                                                                         stop_loss = hp[3], 
                                                                         end_date = is_end_date)
            ret2, _, portfolio_values2, hits2, dates2 = pairs_trade_strategy(merged_df, 
                                                                             cash = 250000 if len(portfolio_values1) == 0 else portfolio_values1[-1],
                                                                             lookback = hp[0], 
                                                                             entry = hp[1], 
                                                                             exit = hp[2], 
                                                                             stop_loss = hp[3],
                                                                             start_date = is_start_date)
            ret = (portfolio_values2[-1] - 250000) / 250000
            portfolio_values = portfolio_values1 + portfolio_values2
            hits = hits1 + hits2
            dates = dates1 + dates2
        else:
            ret1, _, portfolio_values1, hits1, dates1 = pairs_trade_strategy(merged_df, 
                                                                         lookback = hp[0], 
                                                                         entry_cond1_val = hp[1], 
                                                                         entry_cond2_val = hp[1],
                                                                         exit_cond1_val = hp[2],
                                                                         exit_cond2_val = hp[2],
                                                                         stop_loss_cond1 = hp[3],
                                                                         stop_loss_cond2 = hp[3],
                                                                         end_date = is_end_date)
            ret2, _, portfolio_values2, hits2, dates2 = pairs_trade_strategy(merged_df, 
                                                                             cash = 250000 if len(portfolio_values1) == 0 else portfolio_values1[-1],
                                                                             lookback = hp[0], 
                                                                             entry_cond1_val = hp[1], 
                                                                             entry_cond2_val = hp[1],
                                                                             exit_cond1_val = hp[2],
                                                                             exit_cond2_val = hp[2],
                                                                             stop_loss_cond1 = hp[3],
                                                                             stop_loss_cond2 = hp[3],
                                                                             start_date = is_start_date)
            ret = (portfolio_values2[-1] - 250000) / 250000
            portfolio_values = portfolio_values1 + portfolio_values2
            hits = hits1 + hits2
            dates = dates1 + dates2
        
        ret = np.round(ret*100, 2)
        hit_ratio = 0
        max_drawdown = 0
        if hits:
            hit_ratio = np.round(np.mean(hits)*100,2)
            max_drawdown = np.round(calc_max_drawdown(portfolio_values)*100,2)
        if ret > max_ret:
            max_ret = ret
            max_ret_hps = hp
            max_ret_pv = portfolio_values
            max_ret_dates = dates
            max_ret_hr = hit_ratio
            max_ret_dd = max_drawdown
        hp_log.append(f'{hp}: Return: {ret}%, Hit Ratio: {hit_ratio}%, Max Drawdown: {max_drawdown}%\n')
    
    # Store all results in hp_log_v{variant}.txt
    logs = [f'(Lookback, Entry, Exit, Stop-loss)\n',
            f'Best HPs: {max_ret_hps}, Return: {max_ret}%, Hit Ratio: {max_ret_hr}%, Max Drawdown: {max_ret_dd}%\n']
    logs = logs + hp_log 
    
    if not os.path.isdir(f'historical_data/{country}/{adr}/logs/'):
        os.mkdir(f'historical_data/{country}/{adr}/logs/')
    fname = f'historical_data/{country}/{adr}/logs/hp_log_v{version}.txt' 
    f = open(fname, 'w')
    f.writelines(logs)
    f.close()
    
    # Plot best hp and store it
    plot_ret(max_ret_dates, max_ret_pv, country, adr, save = True, filename = f'is_hp_pv_plot_v{version}.png')
    
    best_hps = f'Country: {country}, ADR_Stock: {adr}, HPs: {max_ret_hps}, Return: {max_ret}%, Hit Ratio: {max_ret_hr}%, Max Drawdown: {max_ret_dd}%\n'
    
    # Store best hps in results_v{version}.txt
    if not os.path.isdir('logs/'):
        os.mkdir('logs/')
    
    fname = f'logs/results_v{version}.txt'
    f = open(fname, 'a')
    f.writelines(best_hps)
    f.close()

In [7]:
v1 = Parallel(n_jobs=14, verbose = 5)(delayed(hp_tune)(pairs_trade_v1, '1', country, adr) for (country, adr) in list_pairs)

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   7 out of  28 | elapsed:  2.3min remaining:  6.9min
[Parallel(n_jobs=14)]: Done  13 out of  28 | elapsed:  2.5min remaining:  2.9min
[Parallel(n_jobs=14)]: Done  19 out of  28 | elapsed:  4.2min remaining:  2.0min
[Parallel(n_jobs=14)]: Done  25 out of  28 | elapsed:  4.6min remaining:   32.7s
[Parallel(n_jobs=14)]: Done  28 out of  28 | elapsed:  4.7min finished


In [8]:
v2 = Parallel(n_jobs=14, verbose = 5)(delayed(hp_tune)(pairs_trade_v2, '2', country, adr) for (country, adr) in list_pairs)

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   7 out of  28 | elapsed:  2.6min remaining:  7.8min
[Parallel(n_jobs=14)]: Done  13 out of  28 | elapsed:  2.9min remaining:  3.3min
[Parallel(n_jobs=14)]: Done  19 out of  28 | elapsed:  4.6min remaining:  2.2min
[Parallel(n_jobs=14)]: Done  25 out of  28 | elapsed:  4.9min remaining:   35.5s
[Parallel(n_jobs=14)]: Done  28 out of  28 | elapsed:  5.0min finished


In [9]:
v3a = Parallel(n_jobs=14, verbose = 5)(delayed(hp_tune)(pairs_trade_v3a, '3a', country, adr) for (country, adr) in list_pairs)

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   7 out of  28 | elapsed:  3.4min remaining: 10.1min
[Parallel(n_jobs=14)]: Done  13 out of  28 | elapsed:  3.8min remaining:  4.4min
[Parallel(n_jobs=14)]: Done  19 out of  28 | elapsed:  6.2min remaining:  2.9min
[Parallel(n_jobs=14)]: Done  25 out of  28 | elapsed:  6.8min remaining:   48.9s
[Parallel(n_jobs=14)]: Done  28 out of  28 | elapsed:  6.9min finished


In [10]:
v3b = Parallel(n_jobs=14, verbose = 5)(delayed(hp_tune)(pairs_trade_v3b, '3b', country, adr) for (country, adr) in list_pairs)

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   7 out of  28 | elapsed:  3.6min remaining: 10.9min
[Parallel(n_jobs=14)]: Done  13 out of  28 | elapsed:  4.1min remaining:  4.7min
[Parallel(n_jobs=14)]: Done  19 out of  28 | elapsed:  6.7min remaining:  3.2min
[Parallel(n_jobs=14)]: Done  25 out of  28 | elapsed:  7.3min remaining:   52.8s
[Parallel(n_jobs=14)]: Done  28 out of  28 | elapsed:  7.4min finished


### Consolidate results for each pair

In [11]:
variants = ['1', '2', '3a', '3b']
pairs_trade_strategy = [pairs_trade_v1, pairs_trade_v2, pairs_trade_v3a, pairs_trade_v3b]

results = []
for v in variants:
    fname = f'logs/results_v{v}.txt'
    with open(fname) as f:
        results.append(f.read())

summ = []        
for (country, adr) in list_pairs:
    summ.append(f'Country: {country}, ADR_Stock: {adr}\n')
    for v in range(4):
        ind = results[v].find(f'Country: {country}, ADR_Stock: {adr}')
        end = results[v][ind:].find('\n')
        res = results[v][ind + len(f'Country: {country}, ADR_Stock: {adr}, '):ind + end]
        summ.append(f'Variant {variants[v]}: ' + res + '\n')

fname = f'logs/results_all.txt'
f = open(fname, 'w')
f.writelines(summ)
f.close()

### Sort results for each pair based on return

In [12]:
def sort_res(string):
    ind = string.find('Return: ')
    end = string[ind:].find('%')
    return float(string[ind + 8:ind + end])

for (country, adr) in list_pairs:
    for v in variants:
        fname = f'historical_data/{country}/{adr}/logs/hp_log_v{v}.txt' 
        f = open(fname, 'r')
        res = f.readlines()
        f.close()
        sorted_res = sorted(res[2:], key = sort_res, reverse = True)
        res = res[:2] + sorted_res
        f = open(fname, 'w')
        f.writelines(res)
        f.close()

## Generate IS trade logs

In [13]:
for (country, adr) in list_pairs:
    merged_df = data_processing(country, adr, fx_dict)
    for i in range(4):
        v = variants[i]
        strat = pairs_trade_strategy[i]
        fname = f'historical_data/{country}/{adr}/logs/hp_log_v{v}.txt' 
        f = open(fname, 'r')
        res = f.readlines()
        f.close()
        ind = res[1].find(', Return: ')
        hps = res[1][:ind]
        best_hps = tuple(map(float, hps[11:-1].split(', ')))
        if i < 2:
            ret1, tr1, pv1, hits1, dates1 = strat(merged_df, 
                                                 cash = 250000,
                                                 lookback = int(best_hps[0]), 
                                                 entry = best_hps[1], 
                                                 exit = best_hps[2],
                                                 stop_loss = best_hps[3],
                                                 end_date = is_end_date)
            ret2, tr2, pv2, hits2, dates2 = strat(merged_df, 
                                                 cash = 250000 if len(pv1) == 0 else pv1[-1],
                                                 lookback = int(best_hps[0]), 
                                                 entry = best_hps[1], 
                                                 exit = best_hps[2],
                                                 stop_loss = best_hps[3],
                                                 start_date = is_start_date)
            ret = (pv2[-1] - 250000) / 250000
            tr = tr1 + tr2
            pv = pv1 + pv2
            hits = hits1 + hits2
            dates = dates1 + dates2
            
        else:
            ret1, tr1, pv1, hits1, dates1 = strat(merged_df, 
                                                 cash = 250000,
                                                 lookback = int(best_hps[0]), 
                                                 entry_cond1_val = best_hps[1], 
                                                 entry_cond2_val = best_hps[1],
                                                 exit_cond1_val = best_hps[2],
                                                 exit_cond2_val = best_hps[2],
                                                 stop_loss_cond1 = best_hps[3],
                                                 stop_loss_cond2 = best_hps[3],
                                                 end_date = is_end_date)
            ret2, tr2, pv2, hits2, dates2 = strat(merged_df, 
                                                 cash = 250000 if len(pv1) == 0 else pv1[-1],
                                                 lookback = int(best_hps[0]), 
                                                 entry_cond1_val = best_hps[1], 
                                                 entry_cond2_val = best_hps[1],
                                                 exit_cond1_val = best_hps[2],
                                                 exit_cond2_val = best_hps[2],
                                                 stop_loss_cond1 = best_hps[3],
                                                 stop_loss_cond2 = best_hps[3],
                                                 start_date = is_start_date)
            ret = (pv2[-1] - 250000) / 250000
            tr = tr1 + tr2
            pv = pv1 + pv2
            hits = hits1 + hits2
            dates = dates1 + dates2
            
        ret = np.round(ret*100, 2)
        hit_ratio = 0
        max_drawdown = 0
        if hits:
            hit_ratio = np.round(np.mean(hits)*100,2)
            max_drawdown = np.round(calc_max_drawdown(pv)*100,2)
        
        result = f'Country: {country}\nADR: {adr}\nReturn: {ret}%\nHit Ratio: {hit_ratio}%\nMax Drawdown: {max_drawdown}%\n'
        result += f'\nTrades\n\n'
        
        fname = f'historical_data/{country}/{adr}/logs/is_trade_logs_v{v}.txt' 
        f = open(fname, 'w')
        f.writelines(result)
        f.writelines(tr)
        f.writelines(f'\nHyperparameters\n\n{best_hps}')
        f.close()

## Out-of-sample returns and trade logs

In [15]:
for (country, adr) in list_pairs:
    merged_df = data_processing(country, adr, fx_dict)
    for i in range(4):
        v = variants[i]
        strat = pairs_trade_strategy[i]
        fname = f'historical_data/{country}/{adr}/logs/hp_log_v{v}.txt' 
        f = open(fname, 'r')
        res = f.readlines()
        f.close()
        ind = res[1].find(', Return: ')
        hps = res[1][:ind]
        best_hps = tuple(map(float, hps[11:-1].split(', ')))
        if i < 2:
            ret, tr, pv, hits, dates = strat(merged_df, 
                             cash = 250000,
                             lookback = int(best_hps[0]), 
                             entry = best_hps[1], 
                             exit = best_hps[2],
                             stop_loss = best_hps[3],
                             start_date = oos_start_date, 
                             end_date = oos_end_date)
        else:
            ret, tr, pv, hits, dates = strat(merged_df, 
                             cash = 250000,
                             lookback = int(best_hps[0]), 
                             entry_cond1_val = best_hps[1], 
                             entry_cond2_val = best_hps[1],
                             exit_cond1_val = best_hps[2],
                             exit_cond2_val = best_hps[2],
                             stop_loss_cond1 = best_hps[3],
                             stop_loss_cond2 = best_hps[3],
                             start_date = oos_start_date, 
                             end_date = oos_end_date)
            
        ret = np.round(ret*100, 2)
        hit_ratio = 0
        max_drawdown = 0
        if hits:
            hit_ratio = np.round(np.mean(hits)*100,2)
            max_drawdown = np.round(calc_max_drawdown(pv)*100,2)
            
        plot_ret(dates, pv, country, adr, save = True, filename = f'oos_hp_pv_plot_v{v}.png')
        
        result = f'Country: {country}\nADR: {adr}\nReturn: {ret}%\nHit Ratio: {hit_ratio}%\nMax Drawdown: {max_drawdown}%\n'
        result += f'\nTrades\n\n'
        
        fname = f'historical_data/{country}/{adr}/logs/oos_trade_logs_v{v}.txt' 
        f = open(fname, 'w')
        f.writelines(result)
        f.writelines(tr)
        f.writelines(f'\nHyperparameters\n\n{best_hps}')
        f.close()
        
        fname = 'logs/oos_cum_results.txt'
        f = open(fname, 'a')
        f.writelines(f'{country} {adr} Variant{v}, Return: {ret}%, Hit Ratio: {hit_ratio}%, Max Drawdown: {max_drawdown}%\n')
        f.close()

### Store best variant + HPs to csv

In [17]:
fname = 'logs/results_all.txt'
f = open(fname, 'r')
res = f.readlines()
f.close()

fname = 'logs/oos_cum_results.txt'
f = open(fname, 'r')
oos_res = f.readlines()
f.close()

column_names = ['country', 'adr_stock', 'lookback', 'entry', 'exit', 'stoploss', 'variant', 'is_ret', 
                'is_hr', 'is_max_dd', 'oos_ret', 'oos_hr', 'oos_max_dd']
summ_df = pd.DataFrame(columns = column_names)

for (country, adr) in list_pairs:
    best_ret = -1
    variant = ""
    lookback = 100
    entry = 1
    exit = 0
    stoploss = 3
    hr = 0
    max_dd = 0
    ind = res.index(f'Country: {country}, ADR_Stock: {adr}\n')
    
    for v in range(1,5):
        ret = float(res[ind + v].split('Return: ')[1].split("%")[0])
        
        if ret > best_ret:
            variant = variants[v - 1]
            hp_ind_start = res[ind + v].find('HPs: ')
            hp_ind_end = res[ind + v].find(', Return: ')
            
            hps = res[ind + v][hp_ind_start + 5:hp_ind_end]
            best_hps = hps[1:-1].split(', ')
            lookback = int(best_hps[0])
            entry = float(best_hps[1])
            exit = float(best_hps[2])
            stoploss = float(best_hps[3])
            
            best_ret = ret
            hr_start = res[ind + v].find('%, Hit')
            dd_start = res[ind + v].find('%, Max')
            hr = float(res[ind + v].split("Hit Ratio: ")[1].split("%")[0])
            max_dd = float(res[ind + v].split("Max Drawdown: ")[1].split("%")[0])
            
    for i in range(len(oos_res)):
        if f'{country} {adr} Variant{variant}' in oos_res[i]:
            oos_ret_ind = i
            break
            
    oos_ret = float(oos_res[oos_ret_ind].split("Return: ")[1].split("%")[0])
    oos_hr = float(oos_res[oos_ret_ind].split("Hit Ratio: ")[1].split("%")[0])
    oos_dd = float(oos_res[oos_ret_ind].split("Max Drawdown: ")[1].split("%")[0])
    
    temp_df = pd.DataFrame([[country, adr, lookback, entry, exit, stoploss, variant, best_ret, hr, max_dd, oos_ret, oos_hr, oos_dd]], columns = column_names)
    summ_df = summ_df.append(temp_df, ignore_index = True)

In [18]:
summ_df

,country,adr_stock,lookback,entry,exit,stoploss,variant,is_ret,is_hr,is_max_dd,oos_ret,oos_hr,oos_max_dd
0,Australia,ATHE_ATH,30,1.0,0.5,3.0,3b,5.46,67.11,0.75,0.17,77.78,0.04
1,Australia,GENE_GTG,30,1.0,0.5,3.5,3a,8.24,74.26,0.53,3.28,60.87,0.35
2,Australia,IMMP_IMM,100,1.0,0.5,3.5,3a,38.08,77.27,1.91,4.69,77.27,1.24
3,Australia,IMRN_IMC,60,1.0,0.0,3.5,3b,11.53,84.44,0.53,1.04,84.21,0.18
4,Australia,JHX_JHX,60,1.0,0.5,3.0,3b,11.00,47.33,3.56,0.74,51.52,0.50
5,Australia,KZIA_KZA,100,1.0,-0.5,3.0,3b,15.66,79.10,0.59,-1.70,58.33,1.88
6,Australia,MESO_MSB,30,1.0,-0.5,3.5,3b,84.63,80.23,17.27,12.24,88.00,1.37
7,Australia,PLL_PLL,60,1.0,0.0,3.5,1,31.18,68.97,6.72,2.11,71.43,0.56
8,Australia,WBK_WBC,100,2.0,0.0,3.5,1,23.61,68.75,9.95,2.96,100.00,2.21
9,China,ACH_2600,30,1.5,-0.5,2.5,3a,20.77,64.10,4.03,-0.13,53.33,0.91


In [19]:
summ_df.to_csv('hps.csv', index = False)